# knock86

In [ ]:
m = nn.Dropout(0.3)
input = torch.randn(2, 10)
output = m(input)

print(output)

tensor([[-0.0000,  0.0271, -0.7837, -0.7760, -0.4227, -0.7098, -0.0000, -0.0000,
         -0.0000, -0.5212],
        [-0.0000, -0.0506,  0.0330, -0.6011,  1.3485, -0.1309,  0.5920, -0.9393,
         -0.0000, -2.4764]])


In [ ]:
from torch.nn import functional as F

class CNN(nn.Module):
  def __init__(self, vocab_size, emb_size, padding_idx, output_size, out_channels, kernel_heights, stride, padding, emb_weights=None):
    super().__init__()
    if emb_weights != None:  # 指定があれば埋め込み層の重みをemb_weightsで初期化
      self.emb = nn.Embedding.from_pretrained(emb_weights, padding_idx=padding_idx)
    else:
      self.emb = nn.Embedding(vocab_size, emb_size, padding_idx=padding_idx)
    self.conv = nn.Conv2d(1, out_channels, (kernel_heights, emb_size), stride, (padding, 0))
    self.drop = nn.Dropout(0.3)
    self.fc = nn.Linear(out_channels, output_size)

  def forward(self, x):
    # x.size() = (batch_size, seq_len)
    emb = self.emb(x).unsqueeze(1)
    # emb.size() = (batch_size, 1, seq_len, emb_size)
    conv = self.conv(emb)
    # conv.size() = (batch_size, out_channels, seq_len, 1)
    act = F.relu(conv.squeeze(3))
    # act.size() = (batch_size, out_channels, seq_len)
    max_pool = F.max_pool1d(act, act.size()[2])
    # max_pool.size() = (batch_size, out_channels, 1) -> seq_len方向に最大値を取得
    out = self.fc(self.drop(max_pool.squeeze(2)))
    # out.size() = (batch_size, output_size)
    return out

In [ ]:
# パラメータの設定
VOCAB_SIZE = len(set(word2id.values())) + 1
EMB_SIZE = 300
PADDING_IDX = len(set(word2id.values()))
OUTPUT_SIZE = 4
OUT_CHANNELS = 100
KERNEL_HEIGHTS = 3
STRIDE = 1
PADDING = 1

# モデルの定義
model = CNN(VOCAB_SIZE, EMB_SIZE, PADDING_IDX, OUTPUT_SIZE, OUT_CHANNELS, KERNEL_HEIGHTS, STRIDE, PADDING, emb_weights=weights)

# 先頭10件の予測値取得
for i in range(10):
  X = dataset_train[i]['inputs']
  print(torch.softmax(model(X.unsqueeze(0)), dim=-1)) #unsqueezeでbatch_sizeを1として追加

tensor([[0.2810, 0.2778, 0.2211, 0.2201]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2493, 0.2811, 0.2615, 0.2082]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2804, 0.3009, 0.2599, 0.1588]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2868, 0.2740, 0.2242, 0.2151]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2617, 0.2865, 0.2542, 0.1975]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2226, 0.3255, 0.2520, 0.1999]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2715, 0.2709, 0.2308, 0.2269]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2838, 0.2763, 0.2565, 0.1833]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2529, 0.2881, 0.2480, 0.2111]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2751, 0.2601, 0.2910, 0.1738]], grad_fn=<SoftmaxBackward0>)
